# RSNA-2025 ベースライン推論 (exp0001_baseline)

このノートブックでは、学習済みモデルを使用した推論と提出ファイル生成（参考用）を行います。

- 実験ID: exp0001_baseline
- モデル: 学習済みGradientBoostingClassifier
- 出力: 14ラベルの予測確率

## 主な内容
1. 学習済みモデルのロード
2. 特徴量作成関数の定義
3. 予測関数の実装
4. サンプル予測の実行


In [ ]:
# 0) セットアップ
import os
import sys
import subprocess
from pathlib import Path
import pickle
import json

IN_COLAB = 'google.colab' in sys.modules
print('IN_COLAB =', IN_COLAB)

if IN_COLAB:
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', '-U', 'pip'], check=True)
    subprocess.run([sys.executable, '-m', 'pip', 'install', '-q',
                    'pandas', 'polars', 'seaborn', 'scikit-learn', 'matplotlib', 'gcsfs', 'fsspec'], check=True)
    from google.colab import auth
    auth.authenticate_user()
    os.chdir('/content')
    REPO_URL = 'https://github.com/Kohei-Arita/RSNA-2025.git'
    REPO_DIR = Path('/content/RSNA-2025')
    if not REPO_DIR.exists():
        subprocess.run(['git', 'clone', REPO_URL], check=True)
    os.chdir('/content/RSNA-2025')
    sys.path.insert(0, str(Path.cwd() / 'src'))

GCS_BUCKET = 'rsna2025-prod'
GCS_BASE = f'gs://{GCS_BUCKET}'
print('GCS_BASE =', GCS_BASE)


In [ ]:
# 1) モデルとメタデータのロード
import pandas as pd
import polars as pl
import numpy as np

models_dir = Path('models/exp0001_baseline')

# モデルのロード
with open(models_dir / 'gbm_baseline.pkl', 'rb') as f:
    gbm = pickle.load(f)

# メタデータのロード（特徴量の列名など）
with open(models_dir / 'metadata.json', 'r') as f:
    metadata = json.load(f)

MOD_COLUMNS = metadata['mod_columns']
print("Model loaded successfully")
print(f"Feature columns: {metadata['feature_columns'][:5]}...")
print(f"Modality columns: {MOD_COLUMNS}")


In [ ]:
# 2) ラベル定義（提出用）
ID_COL = 'SeriesInstanceUID'
LABEL_COLS = [
    'Left Infraclinoid Internal Carotid Artery',
    'Right Infraclinoid Internal Carotid Artery',
    'Left Supraclinoid Internal Carotid Artery',
    'Right Supraclinoid Internal Carotid Artery',
    'Left Middle Cerebral Artery',
    'Right Middle Cerebral Artery',
    'Anterior Communicating Artery',
    'Left Anterior Cerebral Artery',
    'Right Anterior Cerebral Artery',
    'Left Posterior Communicating Artery',
    'Right Posterior Communicating Artery',
    'Basilar Tip',
    'Other Posterior Circulation',
    'Aneurysm Present'
]

print(f"Number of labels: {len(LABEL_COLS)}")


In [ ]:
# 3) 訓練データの読み込み（年齢の中央値計算用）
train_uri = f'{GCS_BASE}/train.csv'
train = pd.read_csv(train_uri, storage_options={'token': 'cloud'})

# 年齢の中央値を計算（欠損値埋め用）
df_age_str = train['PatientAge'].astype(str)
age_first = df_age_str.str.split(' - ').str[0]
age_vals = pd.to_numeric(age_first.str.extract(r'([0-9]+(?:\.[0-9]+)?)')[0], errors='coerce')
AGE_MEDIAN = age_vals.median()
print(f"Age median for imputation: {AGE_MEDIAN:.1f}")


In [ ]:
# 4) 特徴量作成関数
def build_feature_row(row):
    """単一行から特徴量を作成"""
    # 年齢の処理
    age_str = str(row.get('PatientAge', ''))
    age_val = pd.to_numeric(age_str.split(' - ')[0], errors='coerce')
    if pd.isna(age_val):
        age_val = AGE_MEDIAN
    
    # 性別の処理
    sex_val = 1 if row.get('PatientSex', '') == 'Male' else 0
    
    # 特徴量辞書を作成
    feats = {'age': age_val, 'sex': sex_val}
    
    # モダリティのone-hot encoding
    modality = row.get('Modality', '')
    for m in MOD_COLUMNS:
        feats[m] = 1 if m == f"mod_{modality}" else 0
    
    return pd.DataFrame([feats])


In [ ]:
# 5) 予測関数
def predict_series(series_data):
    """シリーズデータから14ラベルの予測を生成"""
    # 特徴量を作成
    feat_df = build_feature_row(series_data)
    
    # presence (Aneurysm Present) の予測
    presence_prob = float(gbm.predict_proba(feat_df)[:, 1][0])
    
    # 簡略化：全ラベルに同じ確率を使用
    # 実際の提出では、ラベルごとに専用モデルを使用すべき
    predictions = {label: presence_prob for label in LABEL_COLS}
    
    return predictions


In [ ]:
# 6) サンプル予測（学習データの先頭N件で動作確認）
def build_submission_preview(n=10):
    """学習データの先頭N件で予測を生成（動作確認用）"""
    rows = []
    
    for _, row in train.head(n).iterrows():
        series_id = row[ID_COL]
        predictions = predict_series(row.to_dict())
        
        # 行を作成（SeriesInstanceUID + 14ラベルの予測）
        row_data = [series_id] + [predictions[label] for label in LABEL_COLS]
        rows.append(row_data)
    
    # Polars DataFrameとして返す
    df = pl.DataFrame(rows, schema=[ID_COL] + LABEL_COLS)
    return df

# 動作確認
submission_preview = build_submission_preview(10)
print("Submission preview:")
display(submission_preview.head())


In [ ]:
# 7) 予測結果の保存（サンプル）
outputs_dir = Path('outputs/preds/exp0001_baseline')
outputs_dir.mkdir(parents=True, exist_ok=True)

# CSVとして保存（参考用）
submission_preview.write_csv(outputs_dir / 'sample_predictions.csv')

print(f"Sample predictions saved to: {outputs_dir}")
print(f"Shape: {submission_preview.shape}")
print("\nNote: 本番のKaggle提出では、サービングAPIを使用します。")
print("このCSVは動作確認・ローカルテスト用です。")
